In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
Copy all the files to a local directory, because I don't have the right permissions on RDSF

"""
import os
import yaml
import shutil
import pathlib
from tqdm import tqdm

# Create an output file directory
battery_dir = "data/battery_dbs/"
if not os.path.isdir(battery_dir):
    os.makedirs(battery_dir)

# Find all the files
with open("config.yaml", "r") as stream:
    dirname = "/mnt/z/" + yaml.safe_load(stream)["smartwatch_dbs_dir"]
dirname = pathlib.Path(dirname)
assert dirname.exists()

# Recurse into all "Week X" directories, extracting all .db files
source_files = [file for file in dirname.glob("Week*/**/*.db")]
dest_files = [os.path.join(battery_dir, file.name) for file in source_files]

# Check that they don't all exist
if not all([os.path.isfile(dest) for dest in dest_files]):
    for source, dest in tqdm(zip(source_files, dest_files), total=len(source_files)):
        assert not os.path.isfile(dest), f"Duplicate name {source}"
        shutil.copyfile(source, dest)

100%|██████████| 77/77 [00:28<00:00,  2.70it/s]


In [43]:
"""
Create a csv of p_id, timestamp and battery level

"""
import sqlite3
import pandas as pd


for path in tqdm(dest_files):
    # Get the p_id from the path
    path = pathlib.Path(path)
    p_id = path.name.split("_")[1]

    # Open the db
    conn = sqlite3.connect(path, uri=True)

    # Get the battery level
    try:
        df = pd.read_sql_query("SELECT * FROM Event;", conn)

    except pd.io.sql.DatabaseError as e:
        print(f"{path} failed with {e}")

    # conn.execute("SELECT * FROM Event")
    # battery = c.fetchall()
    # assert len(battery) == 1
    # battery = battery[0][1]

    # # Get the timestamp
    # c.execute("SELECT * FROM battery_timestamp")
    # timestamp = c.fetchall()
    # assert len(timestamp) == 1
    # timestamp = timestamp[0][1]

    # # Write to a csv
    # with open(os.path.join(battery_dir, "battery.csv"), "a") as f:
    #     f.write(f"{p_id},{timestamp},{battery}\n")

    # Close the db
    conn.close()

100%|██████████| 77/77 [00:00<00:00, 1155.03it/s]

data/battery_dbs/EMADATA_15082_20220420.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
data/battery_dbs/EMADATA_20050_20220412.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
data/battery_dbs/EMADATA_29061_20220628.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
data/battery_dbs/EMADATA_4260_20211221.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
data/battery_dbs/EMADATA_27345_20220615.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
data/battery_dbs/EMADATA_35272_20220520.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
data/battery_dbs/EMADATA_3017_20220204.db failed with Execution failed on sql 'SELECT * FROM Event;': database disk image is malformed
data/battery_dbs/EMADATA_23532_20220204.db failed 

In [4]:
"""
Find the start day of each participant

"""

'\nFind the start day of each participant\n\n'

In [5]:
"""
Find the delta for each participant

"""

'\nFind the delta for each participant\n\n'

In [6]:
"""
Plot battery level against delta

"""

'\nPlot battery level against delta\n\n'

In [7]:
"""
Find which participants made 0/few entries on the last day of the study

"""

'\nFind which participants made 0/few entries on the last day of the study\n\n'

In [8]:
"""
Plot the battery level of people who made few entries on the last day

"""

'\nPlot the battery level of people who made few entries on the last day\n\n'

In [9]:
"""
Plot battery level of people who made a lot of entries on the last day

"""

'\nPlot battery level of people who made a lot of entries on the last day\n\n'

In [10]:
"""
Scatter plot showing number of entries per day vs battery level on that day, colour coded by day

"""

'\nScatter plot showing number of entries per day vs battery level on that day, colour coded by day\n\n'